In [5]:
import pandas as pd
import numpy as np
df = pd.read_csv("./MLC_SP23/clean_data/merged_data_v3.csv", low_memory=False, index_col=[0])

In [6]:
df.head()

bbl  total_units_19  year_built  ejectment_count  possession_count  \
0  1000780047             483        2016              0.0               0.0   
1  1001350019               4        1915              0.0               0.0   
2  1001620013              20        1915              0.0               0.0   
3  1001630024              16        1915              0.0               0.0   
4  1001630033              11        1915              0.0               0.0   

   eviction_count  aleration_count  altered_lessthan5yrs  \
0             0.0              0.0                   0.0   
1             0.0              0.0                   0.0   
2             0.0              0.0                   0.0   
3             0.0              0.0                   0.0   
4             0.0              1.0                   0.0   

   altered_morethan5yrs  classanet_avg  ...  boro_3  boro_4  boro_5  \
0                   0.0            0.0  ...       0       0       0   
1                   0.0            0.0  ...       0       0       0   
2                   0.0            0.0  ...       0       0       0   
3                   0.0            0.0  ...       0       0       0   
4                   1.0            0.0  ...       0       0       0   

   rs_share_diff_new  hpd_call_2017_19  hh_inc_avg_19_log1  \
0                0.0               4.0           12.276639   
1                0.0               0.0           12.897523   
2                1.0               2.0           10.973689   
3                0.0               1.0           10.973689   
4                0.0               0.0           10.973689   

   pop_race_asian_pct_19_log1  pop_race_black_pct_19_log1  \
0                    0.150551                    0.018246   
1                    0.096864                    0.055298   
2                    0.461379                    0.096629   
3                    0.461379                    0.096629   
4                    0.461379                    0.096629   

   pop16_unemp_pct_19_log1  pop_mobile1_diff_state_pct_19_log1  
0                 0.035008                            0.074674  
1                 0.002253                            0.014341  
2                 0.037102                            0.008935  
3                 0.037102                            0.008935  
4                 0.037102                            0.008935  

[5 rows x 75 columns]

# Train-test split

In [7]:
X = df.drop(['rs_share_diff_new'], axis = 1)
y = df['rs_share_diff_new']

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [35]:
# Baseline accuracy
1 - y_train.mean()

0.6592301112061592

# Random Forest

In [46]:
# Fit model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import time

scoring = {'Accuracy':'accuracy',
           'F1 Score':'f1',
           'Precision':'precision',
           'Recall':'recall',
           'AUC':'roc_auc'
}

param_grid = {'max_depth':[6, 8, 10],
              'n_estimators': [350, 400, 450, 500]
}

start = time.time()

rf = RandomForestClassifier(random_state = 1)
gr = GridSearchCV(rf, param_grid = param_grid, scoring = scoring, refit = "AUC", cv = 4)
fit = gr.fit(X_train, y_train)

end = time.time()

print("Running grid search took", end - start, "seconds") # time in seconds to run grid search
print()
print(fit.best_params_)
print(fit.best_score_)

Running grid search took 432.40274477005005 seconds

{'max_depth': 10, 'n_estimators': 350}
0.8067566849128225


In [47]:
# get all cv metrics for best model

best_params_index = fit.cv_results_['rank_test_AUC'][0] - 1
print("Best model AUC:", fit.cv_results_['mean_test_AUC'][best_params_index])
print("Best model accuracy:", fit.cv_results_['mean_test_Accuracy'][best_params_index])
print("Best model F1 score:", fit.cv_results_['mean_test_F1 Score'][best_params_index])
print("Best model precision:", fit.cv_results_['mean_test_Precision'][best_params_index])
print("Best model recall:", fit.cv_results_['mean_test_Recall'][best_params_index])

Best model AUC: 0.8067566849128225
Best model accuracy: 0.82600539773931
Best model F1 score: 0.6788421589034366
Best model precision: 0.9145672971794064
Best model recall: 0.5398139766781066


In [48]:
# score model on test set
from sklearn import metrics

y_hat_test = fit.predict(X_test)
auc = metrics.roc_auc_score(y_test, y_hat_test)
acc = metrics.accuracy_score(y_test, y_hat_test)
f1 = metrics.f1_score(y_test, y_hat_test)
prec = metrics.precision_score(y_test, y_hat_test)
rec = metrics.recall_score(y_test, y_hat_test)

print("Out of sample AUC is:", auc)
print("Out of sample accuracy is:", acc)
print("Out of sample F1 score is:", f1)
print("Out of sample precision is:", prec)
print("Out of sample recall is:", rec)

Out of sample AUC is: 0.7572647120144724
Out of sample accuracy is: 0.828679942516937
Out of sample F1 score is: 0.6788531845295364
Out of sample precision is: 0.9074074074074074
Out of sample recall is: 0.5422686750691669


# Parameter tuning history

In [ ]:
# Initial run: max_depth = 6, n_estimators = 100
# 90 seconds

#Out of sample AUC is: 0.7509668635947888
#Out of sample accuracy is: 0.8254978443851365
#Out of sample F1 score is: 0.668357393679282
#Out of sample precision is: 0.9145755472504005
#Out of sample recall is: 0.5265908392253305

In [45]:
# Slight improvement: max_depth = 6, n_estimators = 400
# 248 seconds

#Out of sample AUC is: 0.7534245212534708
#Out of sample accuracy is: 0.8267296243071238
#Out of sample F1 score is: 0.6724873884361661
#Out of sample precision is: 0.911625460284061
#Out of sample recall is: 0.532739010144482

In [ ]:
# Another slight improvement: max_depth = 10, n_estimators = 350
# 432 seconds

#Out of sample AUC is: 0.7572647120144724
#Out of sample accuracy is: 0.828679942516937
#Out of sample F1 score is: 0.6788531845295364
#Out of sample precision is: 0.9074074074074074
#Out of sample recall is: 0.5422686750691669